# `CLOUD.jl` - 2D linear advection

In [1]:
using CLOUD
using Plots
using OrdinaryDiffEq
plots_path = "../plots/advection2d_demo/"
results_path = "../results/advection2d_demo/";

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1342


Define the physical problem

In [2]:
a = (1.0,1.0)  # advection velocity
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = sqrt(2)*L  # end time
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation(a,λ=1.0)
u_exact = initial_condition(initial_data, conservation_law);

Set up a strong-form DGSEM-LGL scheme of degree `p` with `M` elements

In [3]:
M = 8  # number of elements
p = 3  # degree of discretization
reference_approximation=ReferenceApproximation(
    DGMulti(p), Tri())
spatial_discretization = SpatialDiscretization(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    (0.0,L), M), reference_approximation)
results_path = save_project(conservation_law,
     spatial_discretization, initial_data, StrongConservationForm(), 
     (0.0, T), Eager(), results_path, overwrite=true, clear=true);

LoadError: MethodError: no method matching basis(::Tri, ::Int64, ::Tuple{Vector{Float64}, Vector{Float64}})
[0mClosest candidates are:
[0m  basis(::Tri, ::Any, ::Any, [91m::Any[39m) at /Users/tristanmontoya/.julia/packages/NodesAndModes/lJvs5/src/tri_element.jl:127
[0m  basis([91m::Line[39m, ::Any, ::Any) at /Users/tristanmontoya/.julia/packages/NodesAndModes/lJvs5/src/line_element.jl:7
[0m  basis([91m::Quad[39m, ::Any, ::Any, [91m::Any[39m) at /Users/tristanmontoya/.julia/packages/NodesAndModes/lJvs5/src/quad_element.jl:2
[0m  ...

Run the `OrdinaryDiffEq` solver with RK4, taking `n_s` snapshots

In [4]:
n_s = 10
dt_scale=0.1
dt = dt_scale*(L/M)/(a*(2*p+1))
ode_problem = semidiscretize(load_project(results_path)...)
save_solution(ode_problem.u0, 0.0, results_path, 0)
sol = solve(ode_problem, RK4(), adaptive=false, 
    dt=dt, save_everystep=false,
    callback=save_callback(results_path, ceil(Int, T/(dt*n_s))))
save_solution(last(sol.u), last(sol.t), results_path, "final");

LoadError: MethodError: no method matching *(::Tuple{Float64, Float64}, ::Int64)
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  *([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:88
[0m  *([91m::StridedArray{P, N} where N[39m, ::Real) where P<:Dates.Period at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Dates/src/deprecated.jl:44
[0m  ...

Visualize the solution and calculate the integral L<sup>2</sup> error norm

In [5]:
plotter = Plotter(spatial_discretization, plots_path)
error_analysis = ErrorAnalysis(conservation_law, spatial_discretization, results_path)
print("L2 error: ", analyze(error_analysis, last(sol.u), u_exact)[1])
visualize(last(sol.u), plotter,
    "final.pdf", exact_solution=u_exact,
     label="U^h(x,t)", label_exact="U(x,t)")

LoadError: UndefVarError: spatial_discretization not defined

Run a dynamic mode decomposition (truncated to `n_m` modes via POD) on snapshot data

In [6]:
n_m = 3
dmd = DMDAnalysis(results_path,rank=2*n_m)
X, Y = load_snapshots(results_path, dmd.time_steps)
σ, λ, ϕ = analyze(dmd, X, Y)
plot_spectrum(dmd, λ[imag(λ) .> 0]) #only upper half of complex plane due to symmetry

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 /Users/tristanmontoya/.julia/packages/JLD2/DcnTD/src/JLD2.jl:230


LoadError: SystemError: opening file "../results/advection2d_demo/time_steps.jld2": No such file or directory

Plot the first `n_m` DMD mode shapes

In [7]:
mode_plotter = Plotter(spatial_discretization, dmd.path)
plot_modes(dmd, mode_plotter, ϕ[:,imag(λ) .> 0], 1)

LoadError: UndefVarError: dmd not defined